
---

## 📘 What is Pydantic?

Imagine this: You're the backend developer for a hospital management system. You receive thousands of patient records every day. Some records are clean. Some are messy. Some have names where ages should be. Some are missing values.

How do you **automatically clean, validate, and safely use** this data without writing tons of error-checking code?

🎯 You use **Pydantic**.

> **Pydantic** is a Python library that provides **data parsing and validation using Python type hints.**

It ensures the data is:

* **Correct in type** (e.g., int vs str)
* **Valid in content** (e.g., age must be > 0)
* And transforms it into a **structured Python object** you can safely work with.

---

## 🎯 What is the Purpose of Pydantic?

Pydantic helps you:

| Purpose                         | Explanation                                                    |
| ------------------------------- | -------------------------------------------------------------- |
| ✅ Enforce data structure        | Accept only well-formed inputs                                 |
| ✅ Validate data types           | Automatically convert `str` to `int`, or raise errors          |
| ✅ Raise meaningful errors       | Clients get helpful validation error responses                 |
| ✅ Create structured models      | Your functions work with **typed objects**, not random dicts   |
| ✅ Use with FastAPI effortlessly | FastAPI uses Pydantic for all **request and response schemas** |

---

## 🛠️ What Operations Can You Perform Through Pydantic?

| Operation                   | Example                                               |
| --------------------------- | ----------------------------------------------------- |
| ✅ Type conversion           | `"5"` → `5` if `int` expected                         |
| ✅ Field validation          | `min_length`, `max_length`, `ge`, `le`, regex         |
| ✅ Default values            | Fields that auto-fill                                 |
| ✅ Custom validation methods | Validate relationships between fields                 |
| ✅ Model serialization       | Convert to JSON or dict                               |
| ✅ Nested models             | Models inside models (e.g., `Patient` with `Address`) |

---

## 🔥 When Is It Needed?

Pydantic is needed when:

* You're building an **API with request bodies**
* You're working with **external input** (form data, JSON, etc.)
* You want **strongly typed, safe, and validated data**
* You’re defining **request schemas** or **response models** in FastAPI

FastAPI uses Pydantic models for:

* Request bodies
* Query parameter models
* Response formats

---

## 🎯 What 2 Main Problems Does Pydantic Solve?

### 1. ✅ **Type Validation**

* Converts or rejects values based on **expected types**
* If you define a field as `int` and get `"5"` (a string), it converts it.
* If it gets `"five"`, it raises a clean error.

```python
from pydantic import BaseModel

class User(BaseModel):
    age: int

u = User(age="25")  # ✅ Converted to int 25
u = User(age="twenty-five")  # ❌ Raises ValidationError
```

---

### 2. ✅ **Data Validation**

* Beyond just type, you can enforce **constraints**:

  * Length of string
  * Value ranges
  * Custom rules

```python
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str = Field(..., min_length=3)
    price: float = Field(..., ge=0)
```

---

## 💻 Example: Pydantic Model with `Patient`

Let’s simulate a real **healthcare API**:

### 1️⃣ Define a model

```python
from pydantic import BaseModel

class Patient(BaseModel):
    name: str
    age: str
```

### 2️⃣ Create an object of that class

```python
# Simulating input data (e.g., from a POST request)
input_data = {
    "name": "Mahbub Hossain",
    "age": "30"
}

# Pydantic automatically parses and validates
patient_obj = Patient(**input_data)
print(patient_obj)
```

### 🔁 Output:

```python
name='Mahbub Hossain' age='30'
```

Even if the input is a dictionary, you’re now working with a **strong, typed object**.

---

### 3️⃣ Use this object inside a function

```python
def greet_patient(patient: Patient):
    return f"Hello {patient.name}, age recorded as {patient.age}"

print(greet_patient(patient_obj))
```

---

## 🧠 Real Case Scenario

Let’s say you’re creating a FastAPI endpoint to register a patient:

```python
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class Patient(BaseModel):
    name: str
    age: str

@app.post("/register")
def register_patient(patient: Patient):
    return {
        "message": f"Patient {patient.name} (age {patient.age}) registered successfully."
    }
```

### ✅ Test:

Send a POST request with:

```json
{
  "name": "Ayesha Rahman",
  "age": "27"
}
```

### ✅ Output:

```json
{
  "message": "Patient Ayesha Rahman (age 27) registered successfully."
}
```

🎯 All parsed and validated by **Pydantic**, and **automatically shown in Swagger UI**.

---


---

## 🎯 Real-World Scenario: Patient Registration with Medical History

You're building a backend for a **hospital's patient registration system**. Each patient may have:

* Basic info (name, age, gender)
* Emergency contact
* List of previous diagnoses
* Insurance details (optional)

We’ll model this entire structure using **Pydantic**, with real constraints and default values.

---

## ✅ Step-by-Step Breakdown

### 🔧 Step 1: Import Dependencies

```python
from typing import List, Optional
from pydantic import BaseModel, Field
```

---

### 🧩 Step 2: Define Nested Models

#### Emergency Contact Model

```python
class EmergencyContact(BaseModel):
    name: str = Field(..., min_length=3)
    phone: str = Field(..., regex=r'^\+?[0-9]{10,15}$')  # Regex for phone number
    relationship: str
```

#### Diagnosis Model

```python
class Diagnosis(BaseModel):
    condition: str
    diagnosed_date: str  # We'll use string for simplicity (could be date)
    is_chronic: bool
```

#### Insurance Info (Optional)

```python
class Insurance(BaseModel):
    provider: str
    policy_number: str
    valid_till: str
```

---

### 🧬 Step 3: Define the Main `Patient` Model

```python
class Patient(BaseModel):
    name: str = Field(..., min_length=2)
    age: int = Field(..., ge=0, le=130)
    gender: str = Field(..., regex="^(male|female|other)$")
    emergency_contact: EmergencyContact
    medical_history: List[Diagnosis] = []
    insurance: Optional[Insurance] = None
```

---

## 🧪 Step 4: Simulate JSON Input (Like from an API Request)

```python
data = {
    "name": "Mahbub Hossain",
    "age": 30,
    "gender": "male",
    "emergency_contact": {
        "name": "Ayesha Hossain",
        "phone": "+8801712345678",
        "relationship": "Wife"
    },
    "medical_history": [
        {
            "condition": "Diabetes",
            "diagnosed_date": "2018-04-12",
            "is_chronic": True
        },
        {
            "condition": "Flu",
            "diagnosed_date": "2023-01-10",
            "is_chronic": False
        }
    ],
    "insurance": {
        "provider": "GreenLife Insurance",
        "policy_number": "GL123456789",
        "valid_till": "2026-12-31"
    }
}
```

---

### 🧠 Step 5: Create a Pydantic Object

```python
patient = Patient(**data)
print(patient)
```

### 🔎 Output:

```python
name='Mahbub Hossain' age=30 gender='male' emergency_contact=EmergencyContact(name='Ayesha Hossain', ...) medical_history=[Diagnosis(...), Diagnosis(...)] ...
```

Now, you can safely access:

```python
print(patient.medical_history[0].condition)  # "Diabetes"
print(patient.insurance.provider)            # "GreenLife Insurance"
```

---

### 🧪 Step 6: Wrap It in a Function

```python
def register_patient(patient: Patient):
    return {
        "message": f"{patient.name} registered successfully with {len(patient.medical_history)} past conditions.",
        "emergency_contact": patient.emergency_contact.name
    }

print(register_patient(patient))
```

---


---

## 🌱 Part 1: Every Field in a Pydantic Class is Required by Default

### 🧠 Why?

By default, **Pydantic assumes that you want clean, predictable input.** If a field is missing, it’ll raise an error. This is great because it prevents incomplete data from slipping into your system.

### 📦 Real-Life Example:

Let’s say you're building a **User Registration API**.

```python
from pydantic import BaseModel

class User(BaseModel):
    username: str
    email: str
```

If someone sends:

```json
{
  "username": "mahbub"
}
```

You’ll get:

```
ValidationError: 'email' field is missing
```

📌 **Because `email` is not optional, it's mandatory**.

This is great for systems that **depend on complete data**—like account creation, billing, etc.

---

## 🌿 Part 2: Making a Field Optional with `Optional`

Sometimes, you want to **allow users to skip certain data**. Maybe a middle name? Or maybe an optional field like `bio`.

### ✅ Use `Optional`:

```python
from typing import Optional
from pydantic import BaseModel

class User(BaseModel):
    username: str
    email: str
    bio: Optional[str] = None
```

Now the user can send just this:

```json
{
  "username": "mahbub",
  "email": "mahbub@gmail.com"
}
```

...and everything’s fine.

### ⚠️ **Important Rule**: If you use `Optional[type]`, you **must also give a default value** like `= None`. Otherwise, it’s still treated as required.

---

## 🌐 Part 3: Validator Shortcuts - `EmailStr`, `AnyUrl`

Imagine you’re validating a contact form where people submit emails or website URLs.

### Instead of writing custom logic...

```python
from pydantic import BaseModel, EmailStr, AnyUrl

class Contact(BaseModel):
    email: EmailStr
    website: AnyUrl
```

Boom. Now Pydantic will **automatically**:

* 🔐 Check if the email is valid (using proper regex)
* 🌍 Check if the website URL is formatted like `https://something.com`

### 🧪 Try sending:

```json
{
  "email": "notanemail",
  "website": "hello"
}
```

It will raise a **400 Bad Request** with validation errors like:

```
value is not a valid email address
value is not a valid URL
```

✅ **No manual checking needed**!

---

## 🧰 Part 4: The Power of `Field()` in Pydantic

Here comes your secret weapon! Think of `Field()` as a **customizer**. It lets you **control**, **document**, and **validate** fields more deeply.

```python
from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str = Field(..., min_length=3, max_length=100, description="Name of the product")
    price: float = Field(..., gt=0, description="Price must be greater than zero")
```

### 🔍 Let’s break this down:

* `...`: means this field is **required**
* `min_length` / `max_length`: limits for string length
* `gt`: means **greater than** (e.g., price > 0)
* `description`: adds metadata, useful in **OpenAPI docs** (FastAPI auto-generates this!)

---

### 🏷️ Other Useful Parameters of `Field()`

| Parameter    | Purpose                                                  |
| ------------ | -------------------------------------------------------- |
| `default`    | Default value for the field                              |
| `alias`      | Use a different key in the input JSON                    |
| `example`    | Show example in FastAPI docs                             |
| `title`      | Title in docs                                            |
| `const=True` | Field must always have the same value                    |
| `ge`, `le`   | Greater than or equal, less than or equal (numeric only) |
| `regex`      | Apply a custom regex rule to string input                |

### ✅ Example with Metadata:

```python
from pydantic import BaseModel, Field

class Address(BaseModel):
    street: str = Field(..., description="Street address", example="123 Banana St")
    zipcode: str = Field(..., regex=r"^\d{5}$", description="5-digit zip code")
```

When you launch the FastAPI docs (`/docs`), this shows up with tooltips and examples!

---

## 🚨 Part 5: The `strict` Parameter

### 📌 What does `strict=True` do?

It tells Pydantic: **don’t be lenient. Enforce exact data types.**

#### 🤖 Example:

```python
from pydantic import BaseModel, Field

class Order(BaseModel):
    quantity: int = Field(..., strict=True)
```

Now if someone sends:

```json
{
  "quantity": "5"
}
```

⛔ This will fail!

Why? Because "5" is a **string**, not an **int**. Normally, Pydantic will convert "5" → `5`, but with `strict=True`, it **refuses automatic coercion**.

---

### 🛠️ When to use `strict=True`?

* When data comes from **untrusted sources**
* When you want **absolute control** (e.g., financial APIs, audit logs)
* When type coercion could cause **logic bugs**

---
